# Validation tests for DC2 1.2i/p `calexp` and `src`

**Author:** Javier Sánchez

**Date last run:** Oct-24-2018

**Goals:** Make astrometry and photometry quality checks on 1.2i/p single-epoch data

All the plots in this notebook can be made with 1.2p as well. You just have to change the `data_imsim` variable to point to the 1.2p repository (`'/global/cscratch1/sd/desc/DC2/data/Run1.2p/rerun/210918/'`) and the kernel to `desc-stack-run1.2p`

In [ ]:
%pylab inline

In [ ]:
import lsst.daf.persistence
import GCR
import GCRCatalogs
import healpy as hp
import pandas as pd
import astropy.io.fits as fits
from astropy.wcs import WCS
import os
import fitsio
from scipy.stats import binned_statistic

We use sklearn to build the catalog matcher

In [ ]:
from sklearn.neighbors import KDTree

We check the directory where the 1.2i data calexps live

In [ ]:
data_imsim = '/global/cscratch1/sd/desc/DC2/data/Run1.2i/rerun/20181007_h'

In [ ]:
butler = lsst.daf.persistence.Butler(data_imsim)

The convenience function `subset` returns a list with the dictionaries to get a specific subset (in this case a list of all the dictionaries that are used to reference the `src` catalogs using the butler).

In [ ]:
datarefs = butler.subset('src')

In [ ]:
print(len(datarefs.cache))

We have over 32,000 sensor-visits available. We can check which ones have complete sensors or the number of visits per filter.

In [ ]:
from collections import Counter
#Trick from @Yao:
#print(Counter((d['filter'] for d in datarefs.cache)))
print(Counter((d['visit'] for d in datarefs.cache)))

Now, we are going to load the catalog that is going to act as benchmark for our validation tests. In this case, we can use either `dc2_truth_run1.2_static` or `dc2_reference_run1.2`. We can check which other catalogs are available by running `GCRCatalogs.available catalogs`.

In [ ]:
#GCRCatalogs.available_catalogs

In [ ]:
#gc = GCRCatalogs.load_catalog('dc2_truth_run1.2_static')
gc = GCRCatalogs.load_catalog('dc2_reference_run1.2')

Let's see the quantities that these catalogs contain

In [ ]:
gc.list_all_quantities()

We will load position and magnitudes from the benchmark catalog(s)

In [ ]:
# Uncomment the top row (and comment the bottom row) if you want to use the truth catalog instead of the reference catalog
#data = gc.get_quantities(['ra','dec','mag_true_u','mag_true_g','mag_true_r','mag_true_i','mag_true_z','mag_true_y','object_id','star','sprinkled'])
data = gc.get_quantities(['ra','dec','mag_u_lsst','mag_g_lsst','mag_r_lsst','mag_i_lsst','mag_z_lsst','mag_y_lsst','object_id','is_resolved'])

In [ ]:
# We create some columns so we can mix and match the catalogs
data['star']=~data['is_resolved']
for band in ['u','g','r','i','z','y']:
    data['mag_true_%s' %band] = data['mag_%s_lsst' %band]

In [ ]:
from astropy.visualization import ZScaleInterval, ImageNormalize, SqrtStretch
def plot_ref_image(xmin, xmax, ymin, ymax,reference,x,y,x2,y2,mag_true,mag_meas,savename, vmin=-5, vmax=5, show_marker=True):
    """Routine to produce plots of the image in a region from xmin
    to xmax, and ymin to ymax of the reference image
    and annotating the position of three more different catalogs
    (for example input objects, detected stars, and detected galaxies)

    Args:
    ----
        xmin: `float` minimum X position in the chip to be shown
        xmax: `float` maximum X position in the chip to be shown
        ymin: `float` minimum Y position in the chip to be shown
        ymax: `float` maximum Y position in the chip to be shown
        reference: `HDU` HDU containing the image to be analyzed
        x, x2, x3: `float` arrays of X positions to be marked on the image
        y, y2, y3: `float` arrays of Y positions to be marked on the image
        vmin: `float` minimum of the color scale
        vmax: `float` maximum of the color scale
    """
    #interval = ZScaleInterval()
    #norm = ImageNormalize(reference[ymin:ymax,xmin:xmax], interval=interval,stretch=SqrtStretch())
    fig, ax = plt.subplots(ncols=1,figsize=(14,14))
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    if show_marker:
        im0 = ax.scatter(x+1,y+1,c=mag_true,label='True',s=90,marker='x',vmin=vmin,vmax=vmax)
        fig.colorbar(im0, ax=ax, shrink=1,label='mag')
        im1 = ax.scatter(x2+1,y2+1,c=mag_meas,label='imSim',s=90,marker='+',vmin=vmin,vmax=vmax)
    #fig.colorbar(im1, ax=ax, shrink=1,label='mag$_{PSF}$')
    #ax.plot(x2+1,y2+1,'+',c='r',label='ImSim',markersize=12)
    #ax.plot(x3+1,y3+1,'o',c='orange',label='ImSim',markersize=8,fillstyle='none')
    ax.grid()
    plt.legend(loc='best')
    im = ax.imshow(reference[ymin:ymax,xmin:xmax],extent=[xmin,xmax,ymin,ymax],cmap='gray', origin="lower",vmin=vmin,vmax=vmax, interpolation='none')
    fig.colorbar(im, ax=ax, shrink=1,label='Pixel counts [ADU]')
    fig.savefig(savename)

Now we are going to load 189 sensor-visits using the butler. We just choose a subset of the columns, you can check which columns are available in the catalog using `src_cat.schema`

In [ ]:
band = 'r'
ra_true = data['ra']
dec_true = data['dec']
mag_true = data['mag_true_%s' %band]
verbose=False
ra_imsim = []
dec_imsim = []
nchild_imsim = []
mag_k_imsim = []
mag_k_err_imsim = []
flux_k_imsim = []
flux_k_err_imsim = []
mag_sdss = []
mag_sdss_err = []
x_im = []
y_im = []
in_footprint = []
mag_aper = []
mag_psf = []
extendedness= []
isphoto = []
isastro = []
ixx_i = []
iyy_i = []
ixy_i = []
ipsf_xx_i = []
ipsf_yy_i = []
ipsf_xy_i = []
e1_i = []
e2_i = []
icount=0
#visit_arr = [159494,185783,193824]
# The visits in the list below are examples of visits that have all sensors simulated
visit_arr = [32678, 181900, 185783, 191127, 191145, 193824, 197425, 200739, 200747, 204595]
imax = 189
for i, visitId in enumerate(datarefs.cache):
    if ((visitId['filter']==band) & (icount<imax)) & (visitId['visit'] in visit_arr):
        raft = visitId['raftName']
        sensor = visitId['detectorName']
        sensor_index = '%03d' % visitId['detector'] # Kind of a bad way to access the WCS...
        #print(raft,sensor,sensor_index)
        visit = '%08d' % visitId['visit']
        filename = visit+'-'+visitId['filter']+'/'+raft+'/calexp_'+visit+'-'+visitId['filter']+'-'+raft+'-'+sensor+'-det'+sensor_index+'.fits'
        filename = os.path.join(data_imsim,'calexp',filename)
        if (os.path.isfile(filename)) & (icount<imax) & (visitId['visit'] in visit_arr):
            #ref = fits.open(filename)[1].header
            src_cat = butler.get('src',visitId)
            calexp = butler.get('calexp',visitId)
            calib = calexp.getCalib()
            #print(calib.getFluxMag0(),calexp.getVariance())
            calib.setThrowOnNegativeFlux(False)
            w = WCS(calexp.getWcs().getFitsMetadata().toDict())
            #w = WCS(ref)
            #print(w.wcs.crval)
            ra_min = w.wcs.crval[0]-0.2
            ra_max = w.wcs.crval[0]+0.2
            dec_min = w.wcs.crval[1]-0.2
            dec_max = w.wcs.crval[1]+0.2
            mask_true = np.where((ra_true>=ra_min) & (ra_true<=ra_max) & (dec_true>=dec_min) & (dec_true<=dec_max))[0]
            #print('Selected', np.count_nonzero(mask_true))
            x_true, y_true = w.all_world2pix(ra_true[mask_true],dec_true[mask_true],0.,ra_dec_order=True)
            mask_2 = np.where((x_true>0) & (x_true<4001) & (y_true>0) & (y_true<4073))[0]
            in_footprint.append(mask_true[mask_2])
            nchild_imsim.append(src_cat.get('deblend_nChild'))
            ra_imsim.append(np.degrees(src_cat.get('coord_ra')))
            dec_imsim.append(np.degrees(src_cat.get('coord_dec')))
            try:
                mag_k_imsim.append(calib.getMagnitude(src_cat.get('ext_photometryKron_KronFlux_instFlux')))
                mag_k_err_imsim.append(calib.getMagnitude(src_cat.get('ext_photometryKron_KronFlux_instFluxErr')))
                flux_k_imsim.append(src_cat.get('base_PsfFlux_instFlux'))
                flux_k_err_imsim.append(src_cat.get('base_PsfFlux_instFluxErr'))
                mag_sdss.append(calib.getMagnitude(src_cat.get('base_SdssShape_instFlux')))
                mag_sdss_err.append(calib.getMagnitude(src_cat.get('base_SdssShape_instFluxErr')))
                mag_aper.append(calib.getMagnitude(src_cat.get('base_CircularApertureFlux_12_0_instFlux')))
                mag_psf.append(calib.getMagnitude(src_cat.get('base_PsfFlux_instFlux')))
                isphoto.append(src_cat.get('calib_photometry_used'))
                isastro.append(src_cat.get('calib_astrometry_used'))
            except:
                mag_k_imsim.append(calib.getMagnitude(src_cat.get('ext_photometryKron_KronFlux_flux')))
                mag_k_err_imsim.append(calib.getMagnitude(src_cat.get('ext_photometryKron_KronFlux_fluxSigma')))
                flux_k_imsim.append(src_cat.get('base_PsfFlux_flux'))
                flux_k_err_imsim.append(src_cat.get('base_PsfFlux_fluxSigma'))
                mag_sdss.append(calib.getMagnitude(src_cat.get('base_SdssShape_flux')))
                mag_sdss_err.append(calib.getMagnitude(src_cat.get('base_SdssShape_fluxSigma')))
                mag_aper.append(calib.getMagnitude(src_cat.get('base_CircularApertureFlux_12_0_flux')))
                mag_psf.append(calib.getMagnitude(src_cat.get('base_PsfFlux_flux')))
                isphoto.append(src_cat.get('calib_photometry_used'))
                isastro.append(src_cat.get('calib_astrometryUsed'))
            x_im.append(src_cat.get('base_SdssCentroid_x'))
            y_im.append(src_cat.get('base_SdssCentroid_y'))
            extendedness.append(src_cat.get('base_ClassificationExtendedness_value'))
            e1_i.append(src_cat.get('ext_shapeHSM_HsmShapeRegauss_e1'))
            e2_i.append(src_cat.get('ext_shapeHSM_HsmShapeRegauss_e2'))
            ixx_i.append(src_cat.get('base_SdssShape_xx'))
            iyy_i.append(src_cat.get('base_SdssShape_yy'))
            ixy_i.append(src_cat.get('base_SdssShape_xy'))
            ipsf_xx_i.append(src_cat.get('base_SdssShape_psf_xx'))
            ipsf_yy_i.append(src_cat.get('base_SdssShape_psf_yy'))
            ipsf_xy_i.append(src_cat.get('base_SdssShape_psf_xy'))
            icount=icount+1
            if verbose:
                print('Using', visitId)
        else:
            pass
            #print('File ',filename, ' not found')

Since we appended before, we got lists of arrays that we want to make just plain arrays using the trick below (maybe this is not the most efficient way to do it).

In [ ]:
ra_imsim = np.concatenate(np.array(ra_imsim)).ravel()
dec_imsim = np.concatenate(np.array(dec_imsim)).ravel()
nchild_imsim = np.concatenate(np.array(nchild_imsim)).ravel()
mag_k_imsim = np.concatenate(np.array(mag_k_imsim)).ravel()
mag_k_err_imsim = np.concatenate(np.array(mag_k_err_imsim)).ravel()
flux_k_imsim = np.concatenate(np.array(flux_k_imsim)).ravel()
flux_k_err_imsim = np.concatenate(np.array(flux_k_err_imsim)).ravel()
x_im = np.concatenate(np.array(x_im)).ravel()
y_im = np.concatenate(np.array(y_im)).ravel()
in_footprint = np.concatenate(np.array(in_footprint)).ravel()
mag_sdss = np.concatenate(np.array(mag_sdss)).ravel()
mag_sdss_err = np.concatenate(np.array(mag_sdss_err)).ravel()
mag_aper = np.concatenate(np.array(mag_aper)).ravel()
mag_psf = np.concatenate(np.array(mag_psf)).ravel()
extendedness = np.concatenate(np.array(extendedness)).ravel()
isphoto = np.concatenate(np.array(isphoto)).ravel()
isastro = np.concatenate(np.array(isastro)).ravel()
ixx_i = np.concatenate(np.array(ixx_i)).ravel()
iyy_i = np.concatenate(np.array(iyy_i)).ravel()
ixy_i = np.concatenate(np.array(ixy_i)).ravel()
ipsf_xx_i = np.concatenate(np.array(ipsf_xx_i)).ravel()
ipsf_yy_i = np.concatenate(np.array(ipsf_yy_i)).ravel()
ipsf_xy_i = np.concatenate(np.array(ipsf_xy_i)).ravel()
e1_i = np.concatenate(np.array(e1_i)).ravel()
e2_i = np.concatenate(np.array(e2_i)).ravel()

Let's take a look at our data!

In [ ]:
plt.scatter(ra_imsim[::10],dec_imsim[::10],s=0.01)
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')

In [ ]:
# We only use objects in the true catalog that lie within the FOV
ra_true = ra_true[in_footprint]
dec_true = dec_true[in_footprint]
mag_true = mag_true[in_footprint]

We are going to show a cutout of the 1.2i images and compare the input and output centroids and magnitudes.

In [ ]:
mask = (nchild_imsim==0) & (ra_imsim>=54.7) & (ra_imsim<=55.6) & (dec_imsim>=-30.2) & (dec_imsim<=-29.4) & (mag_sdss>10) & (mag_sdss<20) 
mask_true = (ra_true>=54.7) & (ra_true<=55.6) & (dec_true>=-30.2) & (dec_true<=-29.4) & (mag_true>10) & (mag_true<30)
try: 
    mag_test = calib.getMagnitude(src_cat.get('base_PsfFlux_instFlux'))
except:
    mag_test = calib.getMagnitude(src_cat.get('base_PsfFlux_flux'))
mask = mag_test < 25
x, y = w.all_world2pix(np.degrees(src_cat.get('coord_ra'))[mask],np.degrees(src_cat.get('coord_dec'))[mask],0.,ra_dec_order=True)
ra_min = w.wcs.crval[0]-0.3
ra_max = w.wcs.crval[0]+0.3
dec_min = w.wcs.crval[1]-0.3
dec_max = w.wcs.crval[1]+0.3
mask_true2 = np.where((ra_true>=ra_min) & (ra_true<=ra_max) & (dec_true>=dec_min) & (dec_true<=dec_max) & (mag_true>0) & (mag_true<25))[0]
x_true, y_true = w.all_world2pix(ra_true[mask_true2],dec_true[mask_true2],0.,ra_dec_order=True)
plot_ref_image(600,1000,0,600,calexp.getImage().array,x_true,y_true,x,y,mag_true[mask_true2],mag_test[mask],'test',vmin=-25,vmax=30,show_marker=True)

Now that we have loaded the input and output catalogs, we need to relate them. One way is via spatial matching using nearest neighbors

In [ ]:
def spatial_closest_mag_1band(ra_data,dec_data,mag_data,
                              ra_true,dec_true,mag_true,true_id,
                              rmax=3,max_deltamag=1.):
    """
    Function to return the closest match in magnitude within a user-defined radius within certain
    magnitude difference.
    
    ***Caveats***: This method uses small angle approximation sin(theta)
    ~ theta for the declination axis. This should be fine to find the closest
    neighbor. This method does not use any weighting.
    
    Args:
    -----
    
    ra_data: Right ascension of the measured objects (degrees).
    dec_data: Declination of the measured objects (degrees).
    mag_data: Measured magnitude of the objects.
    ra_true: Right ascension of the true catalog (degrees).
    dec_true: Declination of the true catalog (degrees).
    mag_true: True magnitude of the true catalog.
    true_id: Array of IDs in the true catalog.
    rmax: Maximum distance in number of pixels to perform the query.
    max_deltamag: Maximum magnitude difference for the match to be good.
    
    Returns:
    --------
    
    dist: Distance to the closest neighbor in the true catalog. If inputs are
    in degrees, the returned distance is in arcseconds.
    true_id: ID in the true catalog for the closest match.
    matched: True if matched, False if not matched.
    """
    X = np.zeros((len(ra_true),2))
    X[:,0] = ra_true
    X[:,1] = dec_true
    tree = KDTree(X,metric='euclidean')
    Y = np.zeros((len(ra_data),2))
    Y[:,0] = ra_data
    Y[:,1] = dec_data
    ind,dist= tree.query_radius(Y,r=rmax*0.2/3600,return_distance=True)
    matched = np.zeros(len(ind),dtype=bool)
    ids = np.zeros(len(ind),dtype=true_id.dtype)
    dist_out = np.zeros(len(ind))
    for i, ilist in enumerate(ind):
        if len(ilist)>0:
            dmag = np.fabs(mag_true[ilist]-mag_data[i])
            good_ind = np.argmin(dmag)
            ids[i]=true_id[ilist[good_ind]]
            dist_out[i]=dist[i][good_ind]
            if np.min(dmag)<max_deltamag:
                matched[i]=True
            else:
                matched[i]=False
        else:
            ids[i]=-99
            matched[i]=False
            dist_out[i]=-99.
    return dist_out*3600., ids,matched

In [ ]:
dd, ind_mag, matched = spatial_closest_mag_1band(ra_imsim[nchild_imsim==0],dec_imsim[nchild_imsim==0],mag_k_imsim[nchild_imsim==0],
                              ra_true[mag_true<26],dec_true[mag_true<26],mag_true[mag_true<26],np.arange(np.count_nonzero(mag_true<26)),
                              rmax=5,max_deltamag=1)

In [ ]:
_, ind_true, matched = spatial_closest_mag_1band(ra_imsim[nchild_imsim==0],dec_imsim[nchild_imsim==0],mag_k_imsim[nchild_imsim==0],
                              ra_true[mag_true<26],dec_true[mag_true<26],mag_true[mag_true<26],data['object_id'],
                              rmax=5,max_deltamag=1)

Almost 200,000 objects have been matched

In [ ]:
len(ind_mag)

The `calexps` contain some objects used for the photometric calibration. We are going to check the photometric residuals in these objects (`isphoto==True`).

In [ ]:
plt.hist(mag_psf[nchild_imsim==0][(matched) & (isphoto[nchild_imsim==0]) &
                                (data['star'][in_footprint][mag_true<26][ind_mag]==1)]-mag_true[mag_true<26][ind_mag][(matched) & (isphoto[nchild_imsim==0]) &
                                                                                                                             (data['star'][in_footprint][mag_true<26][ind_mag]==1)],range=(-0.1,0.1),bins=100, histtype='step',label='stars')
plt.hist(mag_psf[nchild_imsim==0][(matched) & (isphoto[nchild_imsim==0]) & 
                                   (data['star'][in_footprint][mag_true<26][ind_mag]==0)]-mag_true[mag_true<26][ind_mag][(matched) & 
                                                                                                                                (isphoto[nchild_imsim==0]) & (data['star'][in_footprint][mag_true<26][ind_mag]==0)],range=(-0.1,0.1),bins=100, histtype='step',label='galaxies')
plt.xlabel('mag$_{PSF}$-mag$_{true}$',fontsize=14)
plt.ylabel('Number of objects',fontsize=14)
plt.legend(loc='best')

The same happens for astrometry. We check the astrometric residuals for objects that have `isastro==True`.

In [ ]:
plt.hist(3600000*(ra_imsim[nchild_imsim==0][(matched) & (isastro[nchild_imsim==0]) &
                                (data['star'][in_footprint][mag_true<26][ind_mag]==1)]-ra_true[mag_true<26][ind_mag][(matched) & (isastro[nchild_imsim==0]) &
                                                                                                                             (data['star'][in_footprint][mag_true<26][ind_mag]==1)]),range=(-200,200),bins=200, histtype='step',label='RA stars')
plt.hist(3600000*(ra_imsim[nchild_imsim==0][(matched) & (isastro[nchild_imsim==0]) & 
                                  (data['star'][in_footprint][mag_true<26][ind_mag]==0)]-ra_true[mag_true<26][ind_mag][(matched) & 
                                                                                                                                (isastro[nchild_imsim==0]) & (data['star'][in_footprint][mag_true<26][ind_mag]==0)]),range=(-200,200),bins=200, histtype='step',label='RA galaxies')
plt.hist(3600000*(dec_imsim[nchild_imsim==0][(matched) & (isastro[nchild_imsim==0]) &
                                (data['star'][in_footprint][mag_true<26][ind_mag]==1)]-dec_true[mag_true<26][ind_mag][(matched) & (isastro[nchild_imsim==0]) &
                                                                                                                             (data['star'][in_footprint][mag_true<26][ind_mag]==1)]),range=(-200,200),bins=200, histtype='step',label='DEC stars')
plt.hist(3600000*(dec_imsim[nchild_imsim==0][(matched) & (isastro[nchild_imsim==0]) & 
                                   (data['star'][in_footprint][mag_true<26][ind_mag]==0)]-dec_true[mag_true<26][ind_mag][(matched) & 
                                                                                                                                (isastro[nchild_imsim==0]) & (data['star'][in_footprint][mag_true<26][ind_mag]==0)]),range=(-200,200),bins=200, histtype='step',label='DEC galaxies')
plt.xlim(-50,50)
plt.xlabel(r'$\Delta X $ [mas]',fontsize=14)
plt.ylabel('Number of objects',fontsize=14)
plt.legend(loc='best')

Now, we are going to check the photometry for all detected objects matched to stars.

In [ ]:
mask_mag = (np.isnan(mag_psf[nchild_imsim==0])==False) & (matched) & (data['star'][in_footprint][mag_true<26][ind_mag]==True) # & (extendedness[nchild_imsim==0]==0)
#mask_mag = (np.isnan(mag_psf[nchild_imsim==0])==False) & (matched) # & (extendedness[nchild_imsim==0]==0)

In [ ]:
good_true = (mag_true<26) & (data['star'][in_footprint]==True) # & (data['is_agn'][in_footprint]==False)
#good_true = (mag_true<26)

In [ ]:
mean_snr, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],flux_k_imsim[nchild_imsim==0][mask_mag]/flux_k_err_imsim[nchild_imsim==0][mask_mag],range=(10,30),bins=50)
mean_im, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],mag_psf[nchild_imsim==0][mask_mag]-mag_true[mag_true<26][ind_mag][mask_mag],range=(10,30),bins=50, statistic='median')
std_im, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],mag_psf[nchild_imsim==0][mask_mag]-mag_true[mag_true<26][ind_mag][mask_mag],range=(10,30),bins=50,statistic='std')
n_im, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],mag_psf[nchild_imsim==0][mask_mag]-mag_true[mag_true<26][ind_mag][mask_mag],range=(10,30),bins=50,statistic='count')
n_true, be, _ = binned_statistic(mag_true[good_true],mag_true[good_true],range=(10,30),bins=50,statistic='count')

In [ ]:
plt.errorbar(0.5*be[1:]+0.5*be[:-1],mean_im,std_im/np.sqrt(n_im),fmt='o',color='red')
plt.hexbin(mag_true[mag_true<26][ind_mag][mask_mag],mag_k_imsim[nchild_imsim==0][mask_mag]-mag_true[mag_true<26][ind_mag][mask_mag],gridsize=200,extent=[14,26,-0.5,0.5])
plt.xlabel('mag$_{true}$',fontsize=16)
plt.ylabel('mag$_{PSF}$-mag$_{true}$',fontsize=16)
plt.colorbar(label='Objects/bin')
plt.grid()
plt.ylim(-0.1,0.1)
plt.xlim(14,26)

Now, we are going to check the sources' sizes as a function of their magnitude and look for the presence of brighter-fatter effect.

In [ ]:
mean_im_t, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],0.2**2*(ixx_i[nchild_imsim==0][mask_mag]+iyy_i[nchild_imsim==0][mask_mag]),range=(10,30),bins=50, statistic='median')
std_im_t, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],0.2**2*(ixx_i[nchild_imsim==0][mask_mag]+iyy_i[nchild_imsim==0][mask_mag]),range=(10,30),bins=50,statistic='std')
n_im_t, be, _ = binned_statistic(mag_true[mag_true<26][ind_mag][mask_mag],0.2**2*(ixx_i[nchild_imsim==0][mask_mag]+iyy_i[nchild_imsim==0][mask_mag]),range=(10,30),bins=50,statistic='count')

In [ ]:
plt.scatter(mag_true[mag_true<26][ind_mag][mask_mag],0.2**2*(ixx_i[nchild_imsim==0][mask_mag]+iyy_i[nchild_imsim==0][mask_mag]),c='b',s=0.4,alpha=0.2,label='stars')
plt.scatter(mag_true[mag_true<26][ind_mag][~mask_mag],0.2**2*(ixx_i[nchild_imsim==0][~mask_mag]+iyy_i[nchild_imsim==0][~mask_mag]),c='r',s=0.4, alpha=0.2,label='galaxies')
plt.errorbar(0.5*be[1:]+0.5*be[:-1],mean_im_t,std_im_t/np.sqrt(n_im_t),fmt='o',c='orange',label='stars median')
plt.ylabel('$T$ [arcsec$^{2}$]',fontsize=16)
plt.xlabel('mag$_{r,true}$',fontsize=16)
plt.grid()
plt.ylim(0.,1.)
plt.xlim(14,24)
plt.legend(loc='best')

Let's zoom in!

In [ ]:
bc = 0.5*(be[1:]+be[:-1])
plt.errorbar(bc,mean_im_t-np.nanmean(mean_im_t[(bc>20) & (bc<22)]),std_im_t/np.sqrt(n_im_t),fmt='o')
plt.ylabel(r'$\Delta T$ [arcsec$^{2}$]',fontsize=16)
plt.xlabel('mag$_{r,true}$',fontsize=16)
plt.grid()
plt.ylim(-0.01,0.01)

We also want to check the overall astrometric quality for all sources. Let's take a look

In [ ]:
plt.hist(3600000*(ra_imsim[nchild_imsim==0][mask_mag]-ra_true[mag_true<26][ind_mag][mask_mag]),bins=500,histtype='step',label='RA')
plt.hist(3600000*(dec_imsim[nchild_imsim==0][mask_mag]-dec_true[mag_true<26][ind_mag][mask_mag]),bins=500,histtype='step',label='DEC')
#plt.plot(np.ones(3)*np.median(3600000*(ra_imsim[nchild_imsim==0][mask_mag]-ra_true[mag_true<26][ind_mag][mask_mag])),np.linspace(0,100,3))
plt.xlabel(r'$\Delta X$ [mas]')
plt.xlim(-100,100)
#plt.ylim(0,80)
plt.legend(loc='best')

Since we are not at the equator, we expect the `RA` and `DEC` distributions to be different.

And let's check the overall magnitude differences:

In [ ]:
plt.hist(mag_psf[nchild_imsim==0][mask_mag]-mag_true[mag_true<26][ind_mag][mask_mag],bins=500,histtype='step');
plt.xlabel('mag$_{PSF}$-mag$_{true}$',fontsize=16)
plt.xlim(-0.25,0.25)

A very useful QA plot is to check the detection efficiency for stars as a function of magnitude:

In [ ]:
plt.errorbar(0.5*(be[1:]+be[:-1]),1.0*n_im/n_true,np.sqrt(n_im+n_true)/n_true,fmt='o')
plt.xlabel('mag$_{true}$',fontsize=12)
plt.ylabel('Detection efficiency (stars)',fontsize=12)
plt.grid()

Another useful QA plot is to construct a depth map:

In [ ]:
def get_depth_map(ra,dec,mags,nside=128):
    good = np.logical_or(np.logical_not(np.isnan(ra)),np.logical_not(np.isnan(dec)))
    pix_nums = hp.ang2pix(nside,np.pi/2.-dec[good]*np.pi/180,ra[good]*np.pi/180)
    map_out = np.zeros(12*nside**2)
    for px in np.unique(pix_nums):
        mask = px==pix_nums
        if np.count_nonzero(mask)>0:
            map_out[px]=np.max(mags[mask])
        else:
            map_out[px]=0.
    return map_out

In [ ]:
def make_hp_map(ra,dec,nside=128):
    good = np.logical_or(np.logical_not(np.isnan(ra)),np.logical_not(np.isnan(dec)))
    pix_nums = hp.ang2pix(nside,np.pi/2.-dec[good]*np.pi/180,ra[good]*np.pi/180)
    pix_counts = np.bincount(pix_nums,minlength=12*nside**2)
    return pix_counts

In [ ]:
test_map = get_depth_map(ra_imsim[~np.isnan(mag_sdss)],dec_imsim[~np.isnan(mag_sdss)],mag_sdss[~np.isnan(mag_sdss)],nside=1024)

In [ ]:
hp.gnomview(test_map, rot=(56, -30), title='Depth', reso=1.3, min=23, max=25, unit=r'5-$\sigma$ depth')

In [ ]:
plt.hist(test_map[test_map>0],range=(22,28),bins=100, histtype='step')
plt.xlabel(r'5-$\sigma$ depth', fontsize=16)